In [ ]:
# Plots for section 4.3 (mv and technology operation)

In [ ]:
# TODO

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from datetime import date, datetime, time, timedelta

# imported own functions
from utils import market_values, market_values_by_time_index, nodal_balance, capacity, capacity_links, \
    capacity_storage_units, get_condense_sum, nodal_balance, generation, generation_links, \
    generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse, resistive_heater, gas_boiler, heat_pump,\
    water_tanks_charger, water_tanks_discharger, solar_thermal

# general variables
font1 = {'fontname': 'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/4.3_systems_mv'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]
markers = ["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]

In [ ]:
# Notebook Functions

def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


In [ ]:
# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

## CALC

In [ ]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units
for n in [stst, exp]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() #13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        # ???????generation in GWh (does this have to be multiplied by 3???????
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MWh?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)

    # function for carriers in n.links.carrier.unique() #53
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) * 2920)

    # function for carriers in n.storage_units.carrier.unique() #2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (capacity_storage_units(n, carrier) * 2920)

    # set market values to nan where generation in corresponding region is lower than % quantile
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        index = df_regions_onshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index,
                                  columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == stst:
        df_stst_ons = df_regions_onshore
        df_stst_off = df_regions_offshore

    if n == exp:
        df_exp_ons = df_regions_onshore
        df_exp_off = df_regions_offshore

df_stst_ons.head()

In [ ]:
#assert 0

## General (4.3.1)

In [ ]:
# investigate in carriers with most generation for both networks
# without grid ("electricity distribution grid", "DC", "H2 pipeline", "H2 pipeline retrofitted")
# storage technologies:

carriers_el_gen = [
    "electricity distribution grid",
    "onwind",
    "solar",
    "BEV charger",
    "offwind-dc",
    "ror",
    "hydro",
    "solar rooftop",
    "offwind-ac",
    "battery charger",
    "battery discharger",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "OCGT",
    "home battery charger",
    "home battery discharger",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DC"
]

carriers_el_con = [
    "PHS",
    "electricity distribution grid",
    "H2 Electrolysis",
    "BEV charger",
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central resistive heater",
    "battery charger",
    "battery discharger",
    "V2G",
    "residential urban decentral resistive heater",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "services rural resistive heater",
    "home battery charger",
    "home battery discharger",
    "DC",
    "DAC",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell",
    "Sabatier",
    "Fischer-Tropsch"
]

carriers_heat_gen1 = [
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central gas boiler",
    "urban central resistive heater",
    "residential urban decentral gas boiler",
    "urban central water tanks charger",
    "urban central water tanks discharger",
    "residential urban decentral resistive heater",
    "services urban decentral gas boiler",
    "urban central solar thermal",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "residential rural gas boiler",
    "services rural resistive heater",
    "services rural gas boiler",
    "services rural water tanks charger",
    "residential rural water tanks charger",
    "services rural water tanks discharger",
    "residential rural water tanks discharger",
    "services urban decentral water tanks charger",
    "residential urban decentral water tanks charger",
    "services urban decentral water tanks discharger",
    "residential urban decentral water tanks discharger",
    "residential urban decentral solar thermal",
    "services urban decentral solar thermal",
    "residential rural solar thermal",
    "services rural solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
"urban central water tanks discharger",
"services rural water tanks charger",
"residential rural water tanks charger",
"services rural water tanks discharger",
"residential rural water tanks discharger",
"services urban decentral water tanks charger",
"residential urban decentral water tanks charger",
"services urban decentral water tanks discharger",
"residential urban decentral water tanks discharger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

In [ ]:
n.links[n.links.carrier == "DAC"]

In [ ]:
all_c = n.generators.carrier.unique().tolist() + n.links.carrier.unique().tolist() + n.storage_units.carrier.unique().tolist()

# carriers not in any list:
set(all_c).difference(carriers_el_gen + carriers_el_con + carriers_h2_gen + carriers_h2_con + carriers_heat_gen1 + carriers_heat_con)

In [ ]:
# Check second and third output of links
# what is with DAC?

In [ ]:
# select carriers which make most sense

# ordered according to (df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum() + df_exp_ons[[c + "_gen" for c in c_el_gen_s]].sum()).sort_values(ascending=False)
# overall less than 50 TWh: urban central gas CHP CC, fuel cell, home battery discharger, OCGT, urban central solid bioass CHP,
c_el_gen_s = [
    "onwind",
    "solar",
    "offwind-dc",
    "hydro",
    "solar rooftop",
    "battery discharger",
    "offwind-ac",
    "ror",
    "urban central gas CHP",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# overall less than 50 TWh: home battery charger
# ordered according to (df_stst_ons[[c + "_con_el" for c in c_el_con_s]].sum() + df_exp_ons[[c + "_con_el" for c in c_el_con_s]].sum()).sort_values(ascending=False)
c_el_con_s = [
    "H2 Electrolysis",
    "BEV charger",
    "battery charger",
    "urban central air heat pump",
    "urban central resistive heater",
    "PHS",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell"
    "Sabatier",
    "Fischer-Tropsch"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# use urban central gas boiler (max generation)
# use urban central water tanks charger (max generation)
# use urban central solar thermal (max generation)

carriers_heat_gen1 = [
    "urban central air heat pump"
    "urban central gas boiler"
    "urban central resistive heater"
    "urban central water tanks discharger"
    "urban central solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

#### MV across regions

In [ ]:
# electricity generating technologies

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[c + "_mv" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[c + "_mv" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_mv" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_mv" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("$€/MWh_{el}$")
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'])

fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_both_box.png")

In [ ]:
# exclude technologies with less than 50 TWh of overall generation in both scenarios:
stst_car = df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 50*1e3].index
exp_car   = df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 50*1e3].index
set(stst_car).intersection(exp_car)

In [ ]:
# Further investigation:

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean()
df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean()

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean() - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean()
(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean() - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean()) / df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean()

(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["75%"] - df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["25%"]) - (df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["75%"] - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["25%"])

(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["std"] - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["std"])

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean().sort_values()
df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean().sort_values()

In [ ]:
# electricity consuming technologies
c_el_con_s

In [ ]:
# consumed electric energy and prices payed
# AC and low voltage are the main buses for electricity (why the difference?) Investigate in differences in lmps?

for n , df in zip([stst, exp], [df_stst_ons, df_exp_ons]):

    for c in c_el_con_s:

        # links
        if c in n.links.carrier.unique().tolist():
            # check if bus 0 is AC or low voltage bus
            if n.links[n.links.carrier == c].bus0.map(n.buses.carrier).unique() in ["AC", "low voltage"]:

                # consumption of link at bus 0
                con = n.links_t.p0.loc[:, n.links.carrier == c]
                con.columns = con.columns.map(n.links.bus0)
                # save consumption per location to df
                con_sum = con.sum()
                con_sum.index = con_sum.index.map(n.buses.location)
                # convert to TWh
                df[f"{c}_con_el"] = con_sum / 1000 * 3
                # get lmp of buses where the links consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                cost_mv.index = cost_mv.index.map(n.buses.location)
                # save cost_mv to df (€/MWH_el)
                df[f"{c}_cost_mv_el"] = cost_mv

        # storage unit: only PHS can store
        elif c in n.storage_units.carrier.unique().tolist():

                # consumption of su
                con = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c]
                con.columns = con.columns.map(n.storage_units.bus)
                # save consumption per location to df in TWh
                df[f"{c}_con_el"] = con.sum() / 1000 * 3
                # get lmp of buses where the su consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                # save cost_mv to df
                df[f"{c}_cost_mv_el"] = cost_mv

        else:
            print(f"{c} not found!")

        # set pries to nan where generation in corresponding region is lower than % quantile
        qt = 0.2
        index = df[f"{c}_con_el"] <= np.nanquantile(df[f"{c}_con_el"], qt)
        df[f"{c}_cost_mv_el"][index] = np.nan

In [ ]:
# exclude ones with less than 50 TWh consumption in both scenarios: home battery charger
#df_exp_ons["H2 Electrolysis_con_el"].sum() < 50*1e3

In [ ]:
# electricity consuming technologies

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_con = df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]]
stst_el_con = stst_el_con.values
exp_el_con = df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]]
exp_el_con = exp_el_con.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_con)
filtered_stst = [d[m] for d, m in zip(stst_el_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_con)
filtered_exp = [d[m] for d, m in zip(exp_el_con.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_con_s]
index = c_el_con_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_cost_mv_el" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_cost_mv_el" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("$€/MWh_{el}$")
plt.xticks(rotation=90)
# plt.title("Market values of electricity consuming technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'], loc="upper left")

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/mv_el_con_both_box.png")

In [ ]:
# Further investigation
df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean()
df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean()

df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean() - df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean()

(df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["75%"] -
 df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["25%"]) - (
            df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["75%"] -
            df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["25%"])

(df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["std"] -
 df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["std"])

df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().sort_values()

## Spatial Differences (4.3.2)

### Spatial correlation of mv, gen

In [ ]:
# Spatial correlation of mv

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))

for i, df, ax in zip([0,1], [df_stst_ons, df_exp_ons], axs):

    corr = df[[c + "_mv" for c in c_el_gen_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')

fig.tight_layout(pad=1.5)
plt.show()

fig.savefig(f"{PLOT_DIR}/mv_el_gen_both_corr.png")

In [ ]:
# Spatial correlation of purchasing prices

fig, axs = plt.subplots(ncols=2, figsize=(8, 3))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)
plt.show()

In [ ]:
# Spatial correlation of both

fig, axs = plt.subplots(ncols=2, figsize=(17, 6))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_mv" for c in c_el_gen_s] + [c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n) + " (con)" for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n)+ " (con)" for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)
plt.show()

In [ ]:
# correlation analysis
# what do you want to find?


corr_gini_res = pd.DataFrame(index=c_el_gen_s, columns=[["corr_mv_gen_STST", "cor_mv_gen_EXP", "gini_gen_STST", "gini_gen_EXP", "gini_gen_diff_EXP-STST", "gini_mv_STST", "gini_mv_EXP", "gini_mv_diff_EXP-STST"]])
for c in c_el_gen_s:
    corr_gini_res.loc[c,"corr_mv_gen_STST"] = df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"cor_mv_gen_EXP"] = df_exp_ons[f"{c}_mv"].corr(df_exp_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"gini_gen_STST"] = gini(df_stst_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_STST"] = gini(df_stst_ons[[f"{c}_mv"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_EXP"] = gini(df_exp_ons[[f"{c}_mv"]].dropna().values)

corr_gini_res["gini_gen_diff_EXP-STST"] = corr_gini_res["gini_gen_EXP"].values - corr_gini_res["gini_gen_STST"].values
corr_gini_res["gini_mv_diff_EXP-STST"] = corr_gini_res["gini_mv_EXP"].values - corr_gini_res["gini_mv_STST"].values
corr_gini_res

**Findings:**
- gini: the higher, the more uneven distributed
- generation is more uneven distributed in EXP case
- mv are more evenly distributed in EXP case

### Electricity producing technologies

#### VRE

In [ ]:
# VRE

# market values across regions as map
carriers = ["onwind", "solar","offwind-dc", "ror"]

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
    else:
        df = df_stst_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_vre_STST_map.png")

In [ ]:
# Further investigation
# TODO why is the correlation between mv and cp for ror positive
c_vre = ["onwind", "solar", "offwind-dc", "solar rooftop", "offwind-ac", "ror"]
for c in c_vre:
    print(c)
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_cf"]))

# generation weighted market value
c = "ror"
((df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"].sum()) * df_stst_ons[f"{c}_mv"]).sum()

In [ ]:
# VRE difference

# market values across regions as map
carriers = ["onwind", "solar","offwind-dc", "ror"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_STST-EXP"] = df_stst_off[f"{c}_mv"] - df_exp_off[f"{c}_mv"]
    else:
        df_stst_ons[f"{c}_mv_STST-EXP"] = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_STST-EXP"].max()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]} mv (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()
fig.savefig(f"{PLOT_DIR}/mv_el_gen_vre_STST-EXP_map.png")

#### Storage

In [ ]:
# Storage

# market values across regions as map
carriers = ["hydro", "battery discharger", "V2G", "PHS"]

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
    else:
        df = df_stst_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv_el_gen_store_STST_map.png")

In [ ]:
# Storage difference

# market values across regions as map
carriers = ["hydro", "battery discharger", "V2G", "PHS"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_STST-EXP"] = df_stst_off[f"{c}_mv"] - df_exp_off[f"{c}_mv"]
    else:
        df_stst_ons[f"{c}_mv_STST-EXP"] = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_STST-EXP"].max()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]} mv (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()
fig.savefig(f"{PLOT_DIR}/mv_el_gen_store_STST-EXP_map.png")

In [ ]:
# Storage
# "hydro", "battery discharger", "V2G", "PHS"

carriers = ["hydro", "V2G"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()}, figsize=(18, 8),
                        gridspec_kw={'width_ratios': [1, 1], 'height_ratios': [1]}
                        )
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
    else:
        df = df_stst_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_stor_hydro_V2G_STST_map.png")

In [ ]:
# plot difference

# Storage difference
# "hydro", "battery discharger", "V2G", "PHS"

carriers = ["hydro", "V2G"]
for c in carriers:
    # if one mv is nan the result is nan as well
    df_stst_ons[f"{c}_mv_STST-EXP"] = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()}, figsize=(18, 8),
                        gridspec_kw={'width_ratios': [1, 1], 'height_ratios': [1]}
                        )
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
    else:
        df = df_stst_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_stor_hydro_V2G_STST-EXP_map.png")

In [ ]:
# Further investigation
c = "V2G"
df_stst_ons[[f"{c}_mv",f"{c}_gen"]].sort_values(by=f"{c}_mv")

In [ ]:
df_exp_ons[[f"{c}_mv",f"{c}_gen"]].sort_values(by=f"{c}_mv")

In [ ]:
# difference in mv
pd.concat([df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"], df_stst_ons[f"{c}_gen"],df_exp_ons[f"{c}_gen"]], axis=1).sort_values(by=f"{c}_mv")

In [ ]:
for c in c_el_gen_s:
    print(c)
    mv_diff = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]
    gen_diff = df_stst_ons[f"{c}_gen"] - df_exp_ons[f"{c}_gen"]
    print(mv_diff.corr(gen_diff))

In [ ]:
# Norway analysis
i_no = df_stst_ons.index[df_stst_ons.index.str.contains("NO")]
# generation weighted market value
c = "hydro"
((df_stst_ons.loc[i_no, f"{c}_gen"] / df_stst_ons.loc[i_no, f"{c}_gen"].sum()) * df_stst_ons.loc[i_no,f"{c}_mv"]).sum()
#df_stst_ons.loc[i_no, f"{c}_gen"].sum()

In [ ]:
c = "V2G"
((df_exp_ons.loc[i_no, f"{c}_gen"] / df_exp_ons.loc[i_no, f"{c}_gen"].sum()) * df_exp_ons.loc[i_no,f"{c}_mv"]).sum()
df_exp_ons.loc[i_no, f"{c}_gen"].sum()

In [ ]:
c_vre = ["hydro", "battery discharger", "V2G", "PHS"]
for c in c_vre:
    print(c)
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_cf"]))

In [ ]:
c_vre = ["hydro", "battery discharger", "V2G", "PHS"]
for c in c_vre:
    print(c)
    print(df_exp_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_exp_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_cf"]))

In [ ]:
# generation weighted market value
c = "hydro"
((df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"].sum()) * df_stst_ons[f"{c}_mv"]).sum()

In [ ]:
# generation weighted market value
c = "hydro"
((df_exp_ons[f"{c}_gen"] / df_exp_ons[f"{c}_gen"].sum()) * df_exp_ons[f"{c}_mv"]).sum()

In [ ]:
# Storage: Battery Discharger

carriers = ["battery discharger", "battery discharger"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},figsize=(18, 8),
                        gridspec_kw={'width_ratios': [1, 0.96], 'height_ratios': [1]})
crs = ccrs.EqualEarth()

# same colorbar withds:
vmin = min(np.nanmin(df_stst_ons[f"{carriers[i]}_mv"]), np.nanmin(df_exp_ons[f"{carriers[i]}_mv"]))
vmax = max(np.nanmax(df_stst_ons[f"{carriers[i]}_mv"]), np.nanmax(df_exp_ons[f"{carriers[i]}_mv"]))

for i, ax in enumerate(axs):

    if i == 0:
        df = df_stst_ons
    else:
        df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   vmin=vmin,
                                   vmax=vmax,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    model = 'STST' if i == 0 else 'EXP'
    ax.set_title(f"{carriers[i]} ({model})", fontsize=16, **font1)

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv_el_gen_bat_disch_STST_EXP_map.png")

In [ ]:
#### Peak plants

In [ ]:
# Peak plants

carriers = ["urban central gas CHP", "urban central solid biomass CHP CC"]

fig, axs = plt.subplots(ncols=2, nrows=1,
                        subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_exp_off
    else:
        df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]]}", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_peak_EXP_map.png")

In [ ]:
# Storage difference

# market values across regions as map
carriers = ["urban central gas CHP", "urban central solid biomass CHP CC"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_STST-EXP"] = df_stst_off[f"{c}_mv"] - df_exp_off[f"{c}_mv"]
    else:
        df_stst_ons[f"{c}_mv_STST-EXP"] = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_STST-EXP"].max()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]]} mv (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/mv_el_gen_peak_STST-EXP_map.png")